In [1]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta
from itertools import chain

from datetime import date


sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [2]:
### Function fire_timeline

def concat_subsets(files):
    df = []
    for f in files:
        manyfr = pd.read_csv(f)

        manyfr = gpd.GeoDataFrame(manyfr)

        manyfr.t = manyfr.t.astype("datetime64[ns]")
        df.append(manyfr)
    df = pd.concat(df)
    return(df)

def get_lt(lt_string = "Lt_CA_Boreal_"):
    files = glob.glob("/projects/old_shared/fire_weather_vis/Lightning_analysis/computed_data/" + lt_string +"*.csv")
    return(concat_subsets(files))


def fire_timeline(fireID, 
                  lt,
                  year = '2023',
                  path_region="QuebecGlobalNRT_3571" , 
                  check_last = False, 
                  FWI_source = "station" ):
    
    '''
    '''
    
    ## Read in the largefire file of the fireID
    try:
        fr = fv.load_large_fire(fireID, year = year, path_region= path_region) ## Cluster of 2 fires. 
    except Exception as e:
        print("Fire ID cannot be opened:",fireID)
        print(e)
        return(None)
        ## TO DO Filter? 
            ## VIIRS Static source filter?
            ## WUI filter? 
            
    ## Subset lightning by time and space
    fr = fr.to_crs("3571")
    
    ## TO DO: Figure out which CA ecoregion/province the fire is in and subset lighting by that? 
    print("Not yet subseting spatially beyond quebec. Assuming quebec bounding box")
    
    min_threshold = fr.t.astype('datetime64[ns]').min() - timedelta(days = 10)
    possible_lt = lt[lt.t <= fr.t.min()]
    possible_lt = possible_lt[possible_lt.t >= min_threshold]

    oldest_perim = fr[fr.t == fr.t.max()]
    first_perim = fr[fr.t == fr.t.min()]
    first_perim.geometry = first_perim.buffer(750*2) ## Two viirs pixels???
    join_lt = gpd.sjoin(possible_lt, first_perim, predicate = 'within', how = "inner")
    
    if len(join_lt) == 0:
        join_lt["num_candidates"] = 0
        join_lt["num_strikes"] = len(possible_lt)
        join_lt["num_strikes_10_days"] = len(possible_lt)
    else:
        ## Extract "denominator" or the # of trikes from same period
        denominator = possible_lt[possible_lt.t >= join_lt.t_left.min()]
        denominator = denominator[denominator.t <= join_lt.t_left.max()]
        join_lt["num_candidates"] = len(join_lt)
        join_lt["num_strikes"] = len(denominator)
        join_lt["num_strikes_10_days"] = len(possible_lt)
        
    ## Get distance to individuals ignitions
    # fr["perim_rank"] = fr.t.rank()
    # first_geom = fr[fr.perim_rank == 1].geometry
    # first_geom = first_geom.iloc[0]
    # num_starts = len(first_geom.geoms)
    # for i in range(0, num_starts):
    #     join_lt["dist_start_" + str(i)] = join_lt.distance(first_geom.geoms[i].centroid)
    #     print(fr[fr.perim_rank == 1].to_crs("4326").geometry.iloc[0].geoms[i].centroid)
        
    ## Rank candidate by distance
    # range_geoms = list(range(0, num_starts))
    # string = "dist_start_"
    # columns_dists = [string + str(x) for x in range_geoms]
    # top = len(join_lt) * 1 # Top 100%. Could cut to smaller range
    # dist_bool = join_lt[columns_dists].rank() <= top ## NEED a max distance cutoff. 
    # join_lt["candidate"] = dist_bool.any(axis = 1)
    
    ## Get raw VIIRS pixel timing
    date_string = fr.t.astype("datetime64[ns]").max().strftime("%Y%m%d%p")
    print(date_string)
    raw_obs_times = fv.raw_pixel_times(int(fireID), date_string = date_string, path_region = path_region)
    raw_obs_times = raw_obs_times.reset_index()
    
    ## get station data
    if(FWI_source == "station"):
        print("Assuming Single Quebec Station. 718270-99999.")
        st = pd.read_csv("s3://veda-data-store-staging/EIS/other/station-FWI/19900101.NRT/FWI/718270-99999.linear.HourlyFWIFromHourlyInterpContinuous.csv") ## Corrected record from Robert
        st.HH = st.HH.astype("int")
        st.YYYY = st.YYYY.astype("int")
        st.MM = st.MM.astype("int")
        st.DD = st.DD.astype("int")
        st = fv.date_convert(st)
        
        st_rm = st[["time", "TEMP_C", 'RH_PERC', 'VPD_HPA', 'WDSPD_KPH',
       'PREC_MM', 'SNOWD_M', 'VIS_KM', 'FFMC', 'DMC', 'DC', 'BUI', 'ISI',
       'FWI', 'OBSMINUTEDIFF_TEMP', 'OBSMINUTEDIFF_RH', 'OBSMINUTEDIFF_WDSPD',
       'ISPRECREPORTED', 'OBSMINUTEDIFF_SNOW', 'OBSMINUTEDIFF_VIS']]
        st_rm = st_rm.rename(columns = {"time":"t"})
        #### Subset station data by time. 
        st_rm = st_rm[st_rm.t >= min_threshold]
        st_rm = st_rm[st_rm.t <= fr.t.max()]
        
    else:
        #print("No other FWI extraction method ready. Sorry. ")
        raise Exception("No other FWI extraction method ready. Sorry. ")
    
    ## Do merging of all dfs 
    foo = join_lt[["InterCloud", "t_left", "lat_left", "lon_left", "current_mag", "error_elps", "num_station"]]
    foo = foo.rename(columns = {"t_left":"t", "lat_left":"lat", "lon_left":"lon"})
    foo.t = foo.t.astype('datetime64[ns]')
    raw_obs_times = raw_obs_times.rename(columns={"count": "viirs_pix_count"}) 
    raw_obs_times.t = raw_obs_times.t.astype("datetime64[ns]")
    merged = foo.merge(raw_obs_times, on = ["t"], how = "outer")
        
    fr_rm = fr.rename(columns = {"lat":"lat_centroid", "lon":"lon_centroid"})
    fr_rm.t = fr_rm.t.astype("datetime64[ns]")
    merged = merged.merge(fr_rm, on = ["t"], how = "outer")
    
    merged = merged.merge(st_rm, on = ["t"], how = "outer")
    merged["fireID"] = fireID
    
    return(merged)
    

def lf_ids(year = None, regnm = 'BOREAL_NRT_3571_DPS'):
    
    diroutdata = "s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/"

    if year == None:
        year = date.today().year

    if diroutdata.startswith("s3://"):
        # Can't use glob for S3. Use s3.ls instead.
        import s3fs
        s3 = s3fs.S3FileSystem(anon=False)
        s3path = os.path.join(diroutdata, regnm, str(year), "Largefire")
        fnms = [f for f in s3.ls(s3path)]


    fnms.sort()
    ids = []
    for f in fnms:
        fnm_lts = os.path.basename(f) 
        one_id = fnm_lts[1:-11]
        ids.append(one_id)
    tmp_ids = pd.DataFrame(ids, columns=["ids"])
    tmp_ids = tmp_ids.ids.unique()
    return(tmp_ids)

def unique(list1):
 
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return(unique_list)

def get_listed_ids(quebec_stats):
    newlist = [x.strip('][\n').split(' ') for x in quebec_stats.fireID.unique()]
    newlist = list(chain(*newlist))
    newlist = [x.replace('\n', ' ') for x in newlist]
    newlist = unique(newlist)
    return(newlist)

# def lf_ids(regnm = 'BOREAL_NRT_3571_DPS', year = None):
    
#     diroutdata = "s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/"

#     if year == None:
#         year = date.today().year

#     if diroutdata.startswith("s3://"):
#         # Can't use glob for S3. Use s3.ls instead.
#         import s3fs
#         s3 = s3fs.S3FileSystem(anon=False)
#         s3path = os.path.join(diroutdata, regnm, str(year), "Largefire")
#         fnms = [f for f in s3.ls(s3path)]
#     else:
#         fnms = glob(os.path.join(diroutdata, regnm, str(year), "Largefire"))

#     if len(fnms) > 0:
#         #print("yeah")
#         fnms.sort()
#         ids = []
#         for f in fnms:
#             fnm_lts = os.path.basename(f) ## Can't work, no ordering
#             ids.append(fnm_lts[1:-11])

#     ids = unique(ids)
#     return(ids)
    

In [3]:
lt = get_lt() 

In [4]:
lt = gpd.GeoDataFrame(lt, geometry=gpd.points_from_xy(lt.lon, lt.lat), crs=4326) #4674
lt = lt.to_crs("3571")

In [5]:
     

#tmp = fire_timeline('615', lt = lt, path_region="QuebecGlobalNRT_DPS") #QuebecGlobalNRT_3571

In [6]:
# date_range = pd.date_range(start = "2023-05-01 12:00:00", end = "2023-07-01 12:00:00", freq="12H")
# #date_range_format = datetime.strptime(date_rage, 
# date_snap = date_range.strftime("%Y%m%d%p")

In [7]:
## Get IDs. These IDs come from csvs made by old_shared/fire_weather_vis/Lightning_analysis/snap_prov_lightning.ipynb
# by going through the snapshot files, doing a spatial join, and collecting IDs. 

files = glob.glob("/projects/old_shared/fire_weather_vis/Lightning_analysis/snap_stats//boreal_snapstats*.csv")

fire_stats = concat_subsets(files)

fire_stats.t.max()

### Subsetting fire stats by largefire record, so don't waste time looking for IDs that we haven't got yet. Wait, not worth it, size a bigger thing anyway.
#fire_stats = fire_stats[fire_stats.t < "2023-07-20 12:00:00"]

#fire_stats = pd.read_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/snap_stats/boreal_snapstats_20231024.csv")

Timestamp('2023-08-30 12:00:00')

In [8]:
quebec_stats = fire_stats[fire_stats.prov_name_en == "Quebec"]

tmp_list = get_listed_ids(quebec_stats)

In [9]:
ids_lf = lf_ids()

tmp_list = list(set(tmp_list).intersection(ids_lf))
#tmp_list

In [10]:
from datetime import date
str(date.today().strftime("%Y%m%d"))

'20231031'

In [62]:
# #ids = ['12641', '12690','10896','9346']

# ids = tmp_list

# max_t = "maxT" + str(fire_stats.t.max()) + "_"
# min_t = "minT" + str(fire_stats.t.min()) + "_"
# year = '2023'
# path_region= "BOREAL_NRT_3571_DPS" 
# check_last = False 
# FWI_source = "station" 

# fires = pd.DataFrame()
# for n,i in enumerate(ids, start = 0):
#     try:
#         foo = fire_timeline(i, lt = lt, year = year, path_region= path_region, check_last = False, FWI_source = FWI_source)
#     except Exception as e:
#         print("Error at ID: ",i)
#         print(e)
#         continue
#     ## Extract the period between 
#     fires = pd.concat([fires, foo])
#     #print(fires)
#         #fr_pd = pd.DataFrame(fires, columns=["lat", "lon", "farea", "data_source"])
#     fires.to_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/fwi_timeline_merge/"+"fire_stats_only_718270-99999_" +min_t + max_t + path_region + FWI_source + str(date.today().strftime("%Y%m%d"))+  ".csv")

In [63]:
fire_timeline(fireID = '13165' , lt = lt, year = year, path_region= path_region , check_last = False, FWI_source = FWI_source)

[]
Fire ID cannot be opened: 13165
No objects to concatenate


In [ ]:
# "20230720PM"

# raw_obs_times = fv.raw_pixel_times(int(12641), date_string = "20230720PM", path_region = "BOREAL_NRT_3571_DPS")

In [ ]:
# # import pickle

# fireID = '12641'
# file = open('/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/BOREAL_NRT_3571_DPS/2023/Serialization/20230720PM.pkl', 'rb')

# # dump information to that file
# data = pickle.load(file)

# # close the file
# file.close()

# fireID = int(fireID)

# times = []
# for i in range(0, len(data.fires[fireID].pixels)):
#     #print(i)
#     times.append(data.fires[fireID].pixels[i].datetime)

In [ ]:
#len(data.fires)

In [ ]:
#len(data.fires[fireID].pixels)

In [12]:
fires.fireID.unique()

array(['10373', '12151', '12150', '10580', '8570', '12149', '12201',
       '8562', '12160', '8502', '8644', '8613', '12467', '12655', '13135',
       '8555', '12375', '12276', '12381', '12595', '13021', '12154',
       '10978', '12144'], dtype=object)

In [ ]:
new_fire = fires.groupby("fireID")

In [42]:
group_first_ig = fires[~fires.InterCloud.isna()].groupby("fireID").t.min() #.reset_index()
group_first_ig

fireID
12375   2023-07-05 10:16:09.925
12467   2023-07-01 03:16:05.704
12595   2023-07-05 12:02:10.601
12655   2023-07-05 13:47:56.871
13021   2023-07-05 19:39:55.798
13135   2023-07-05 19:01:33.248
Name: t, dtype: datetime64[ns]

In [43]:
group_first_detect = fires[~fires.viirs_pix_count.isna()].groupby("fireID").t.min() #.reset_index()
group_first_detect

fireID
10373   2023-06-16 17:02:00
10580   2023-06-20 17:28:00
10978   2023-06-22 17:41:00
12144   2023-07-03 05:12:00
12149   2023-07-03 06:03:00
12150   2023-07-03 06:03:00
12151   2023-07-03 06:03:00
12154   2023-07-03 06:03:00
12160   2023-07-03 06:54:00
12201   2023-07-03 16:43:00
12276   2023-07-04 16:26:00
12375   2023-07-05 17:45:00
12381   2023-07-05 16:56:00
12467   2023-07-06 06:50:00
12595   2023-07-07 06:29:00
12655   2023-07-07 18:00:00
13021   2023-07-10 16:13:00
13135   2023-07-11 06:03:00
8502    2023-06-01 16:41:00
8555    2023-06-02 06:35:00
8562    2023-06-02 06:37:00
8570    2023-06-02 06:37:00
8613    2023-06-02 16:24:00
8644    2023-06-02 17:15:00
Name: t, dtype: datetime64[ns]

In [31]:
smol["pre_fire"] = ((smol.t >= last_ig) & (smol.t <=  first_detection))

,fireID,t
0,12375,2023-07-05 10:16:09.925
1,12467,2023-07-01 03:16:05.704
2,12595,2023-07-05 12:02:10.601
3,12655,2023-07-05 13:47:56.871
4,13021,2023-07-05 19:39:55.798
5,13135,2023-07-05 19:01:33.248


In [ ]:
fires.columns

In [ ]:
fires.groupby('fireID').t["Inter"]

In [ ]:
fires.fireID.unique()

In [38]:
smol = fires[fires.fireID == '12375']

In [48]:
subset_group_first_detect = group_first_detect[group_first_detect.index.isin(group_first_ig.index)]

In [50]:
fires.groupby('fireID').t >= group_first_ig

ValueError: Lengths must match

In [40]:
first_ig = smol[~smol.InterCloud.isna()].t.min()
print(first_ig)
last_ig = smol[~smol.InterCloud.isna()].t.max()
print(last_ig)


first_detection = smol[~smol.viirs_pix_count.isna()].t.min()
print(first_detection)

feds_start = smol[~smol.farea.isna()].t.min()
print(feds_start)

### Growht threshold



smol["pre_fire"] = ((smol.t >= last_ig) & (smol.t <=  first_detection)) #### first_ig better????????????


2023-07-05 10:16:09.925000
2023-07-05 10:25:15.219000
2023-07-05 17:45:00
2023-07-05 12:00:00


/tmp/ipykernel_177/162921546.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smol["pre_fire"] = ((smol.t >= last_ig) & (smol.t <=  first_detection)) #### first_ig better????????????


In [27]:
pd.isnull(first_ig)

True

In [ ]:
fires[~fires.viirs_pix_count.isna()].t.unique()

In [ ]:
# fr = fr.sort_values(by = ['t'])

# fig, ax = plt.subplots()
# ax.fill_between(upper.index, upper.FWI.rolling(1).mean(), lower.FWI.rolling(5).mean(), 
#                 facecolor='grey', 
#                 alpha=0.2,
#                 label= "95th Percentile")
# ax.fill_between(mid_upper.index, mid_upper.FWI.rolling(1).mean(), mid_lower.FWI.rolling(5).mean(), 
#                 facecolor='grey', 
#                 alpha=0.4,
#                 label= "25th Percentile")
# ax.plot(mean_quant.index, mean_quant.FWI.rolling(1).mean(), 
#         color = "black",
#         label= "Historic Mean Per Day")
# ax.plot(st[(st.time >= "2023-05-01")].time.astype('datetime64[ns]'), st[(st.time >= "2023-05-01")].FWI)
# ax.set_ylabel("Fire Weather Index")
# ax.set_title("2023 Fire Weather Index (FWI) for La Grande Rivière, Quebec, Canada (WMO ID 718270)")
# #ax.legend()
# ax2 = ax.twinx()
# ax2.scatter(candidate.t_left.astype("datetime64[ns]"), candidate.candidate, color = "purple")
# ax2.set_yticklabels("")
# #ax2.plot(fire_stats.t, fire_stats.num_active_fires, color = "red", label = "Number of Fires in Quebec")
# #ax2.legend(loc = 0.5)
# ax3 = ax.twinx()
# #ax3.spines.right.set_position(("axes", 1.2))
# ax3.plot(fr.t.astype("datetime64[ns]"), fr.farea, color = "red")
# #ax3.plot(day_strike.index.astype("datetime64[ns]"), day_strike.InterCloud, color = "orange")

# ax4 = ax.twinx()
# ax4.spines.right.set_position(("axes", 1.2))
# ax4.scatter(raw_obs_times.index, raw_obs_times['count'], color = "orange")

# fig.autofmt_xdate()

# # print("First detection: " + str(fr.t.astype("datetime64[ns]").min()) )
# print(candidate.t_left.min())
# print(candidate.t_left.max())

In [83]:
def gpd_read_file(filename, parquet=False, **kwargs):
    itry = 0
    maxtries = 5
    fun = gpd.read_parquet if parquet else gpd.read_file
    while itry < maxtries:
        try:
            dat = fun(filename, **kwargs)
            return dat
        except Exception as e:
            itry += 1
            print(f"Attempt {itry}/{maxtries} failed.")
            if not itry < maxtries:
                raise e



def load_large_fire(fireID, year = "2019", path_region = "WesternUS", layer='perimeter', s3_path = False):
    '''
    loads in largefire file based on fireID and layer, then preps it for "explore" by adding centriod data. Currently limited to one year. 
    
    INPUTS:
        
        fireID (str): fireID offire of interest. Can be found in gdf files read in by prep_gdf and load_file. Can be selected interactivly form a gdf if use gdf.explore()
        year (str): Year that fires took place. Default to 2019. Availible options differ by path_region. 
        path_region (str): This constructs the path that the fires are stored in. WesternUS and CONUS availible. 
        layer (str): The largefire layer to load. Options are 'perimeter', 'nfplist', 'fireline', and 'newfirepix'. 
    '''
    if(s3_path == True):
        tmp = s3.glob('s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/' + path_region +'/'+ year +'/Largefire/F' + fireID + '_*')
        lf_files =  ["s3://" + t for t in tmp]
    
    else:
        lf_files = glob.glob('/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/' + path_region +'/'+ year +'/Largefire/F' + fireID + '_*')
        #print(lf_files)
    lf_ids = list(set([file.split('Largefire/')[1].split('_')[0] for file in lf_files])) 
    print(lf_ids)
    largefire_dict = dict.fromkeys(lf_ids)
    
    for lf_id in lf_ids:
        most_recent_file = [file for file in lf_files if lf_id in file][-1]
        largefire_dict[lf_id] = most_recent_file
    if(s3_path == True):
        gdf = pd.concat([gpd_read_file(file,layer= layer) for key, file in largefire_dict.items()], 
                       ignore_index=True)
    else:
        gdf = pd.concat([gpd.read_file(file,layer= layer) for key, file in largefire_dict.items()], 
                       ignore_index=True)
    gdf = gdf.to_crs('EPSG:4326')
    gdf['lon'] = gdf.centroid.x
    gdf['lat'] = gdf.centroid.y
    return gdf

In [84]:
load_large_fire(fireID = '13165', year = '2023', path_region= "BOREAL_NRT_3571_DPS", s3_path = True)

['F13165']


/tmp/ipykernel_177/4002652227.py:49: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/tmp/ipykernel_177/4002652227.py:50: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


,n_pixels,n_newpixels,farea,fperim,flinelen,...,meanFRP,t,geometry,lon,lat
0,77,77,20.093947,34.028644,31.234218,...,1.906494,2023-07-11,"MULTIPOLYGON (((-78.20751 51.02346, -78.20754 ...",-78.276415,51.03515


In [74]:
fireID = '13165'

s3.glob('s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/' + path_region +'/'+ year +'/Largefire/F' + fireID + '_*')

['maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/BOREAL_NRT_3571_DPS/2023/Largefire/F13165_20230711AM']

In [80]:
tmp = s3.glob('s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/' + path_region +'/'+ year +'/Largefire/F' + fireID + '_*')

tmp =  ["s3://" + t for t in tmp]

tmp

['s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/BOREAL_NRT_3571_DPS/2023/Largefire/F13165_20230711AM']